In [2]:
%pip install streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 368.9 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 35.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 5.2 MB/s eta 0:00:00


In [8]:
# Price Prediction Model with Streamlit UI

# --- Step 1: Imports ---
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
from scipy.stats import spearmanr
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import os
from datetime import datetime
import streamlit as st
import tempfile
warnings.filterwarnings("ignore")

# --- Step 1.5: Generate Mock Data if Not Present ---
def generate_mock_csvs():
    if not os.path.exists("data"):
        os.makedirs("data")

    dates = pd.date_range(end=datetime.today(), periods=200)

    def gen_data(base=100):
        return base + np.cumsum(np.random.randn(len(dates)))

    pd.DataFrame({"date": dates, "price_lme": gen_data()}).to_csv("data/lme.csv", index=False)
    pd.DataFrame({"date": dates, "price_jpx": gen_data()}).to_csv("data/jpx.csv", index=False)
    pd.DataFrame({"date": dates, "forex_rate": gen_data(1.1)}).to_csv("data/forex.csv", index=False)
    pd.DataFrame({"date": dates, "stock_index": gen_data(3000)}).to_csv("data/us_stock.csv", index=False)

# --- Step 2: Streamlit App ---
st.set_page_config(page_title=" Price Prediction", layout="wide")
st.title("📈 Devolped by Haider Ali")
st.markdown("Use historical market data to forecast future commodity returns.")

st.markdown("""
> 🔍 **Model Summary** (330 characters):
> This model uses XGBoost to predict next-day returns of LME commodity prices based on market data from JPX, Forex, and US stocks. It includes features like returns, volatility, SMA, momentum, and MACD. XGBoost captures complex patterns for accurate forecasting.
""")

# --- Step 3: Data Upload or Generation ---
if st.button("📁 Generate Sample Data"):
    generate_mock_csvs()
    st.success("Mock CSV files generated!")

st.sidebar.header("Upload Your CSV Files")
lme_file = st.sidebar.file_uploader("Upload LME CSV", type="csv")
jpx_file = st.sidebar.file_uploader("Upload JPX CSV", type="csv")
forex_file = st.sidebar.file_uploader("Upload Forex CSV", type="csv")
stock_file = st.sidebar.file_uploader("Upload US Stock CSV", type="csv")

if lme_file and jpx_file and forex_file and stock_file:
    lme = pd.read_csv(lme_file, parse_dates=['date'])
    jpx = pd.read_csv(jpx_file, parse_dates=['date'])
    forex = pd.read_csv(forex_file, parse_dates=['date'])
    stock = pd.read_csv(stock_file, parse_dates=['date'])
else:
    lme = pd.read_csv("data/lme.csv", parse_dates=['date'])
    jpx = pd.read_csv("data/jpx.csv", parse_dates=['date'])
    forex = pd.read_csv("data/forex.csv", parse_dates=['date'])
    stock = pd.read_csv("data/us_stock.csv", parse_dates=['date'])

# --- Step 4: Merge Data ---
df = lme.merge(jpx, on='date', suffixes=('_lme', '_jpx'))
df = df.merge(forex, on='date')
df = df.merge(stock, on='date')
df.sort_values('date', inplace=True)

# --- Step 5: Feature Engineering ---
df['return_lme'] = np.log(df['price_lme'] / df['price_lme'].shift(1))
df['return_jpx'] = np.log(df['price_jpx'] / df['price_jpx'].shift(1))
df['return_forex'] = np.log(df['forex_rate'] / df['forex_rate'].shift(1))
df['return_stock'] = np.log(df['stock_index'] / df['stock_index'].shift(1))

# Rolling features
df['vol_lme'] = df['return_lme'].rolling(window=5).std()
df['vol_jpx'] = df['return_jpx'].rolling(window=5).std()
df['vol_stock'] = df['return_stock'].rolling(window=5).std()

# Moving averages and indicators
df['sma_gap'] = df['price_lme'].rolling(5).mean() - df['price_lme'].rolling(10).mean()
df['momentum_3'] = df['price_lme'] - df['price_lme'].shift(3)
df['momentum_5'] = df['price_lme'] - df['price_lme'].shift(5)
df['ema_12'] = df['price_lme'].ewm(span=12, adjust=False).mean()
df['ema_26'] = df['price_lme'].ewm(span=26, adjust=False).mean()
df['macd'] = df['ema_12'] - df['ema_26']
df['macd_signal'] = df['macd'].ewm(span=9, adjust=False).mean()
df['target'] = df['return_lme'].shift(-1)
df.dropna(inplace=True)

# --- Step 6: Model Training ---
features = [
    'return_jpx', 'return_forex', 'return_stock',
    'vol_lme', 'vol_jpx', 'vol_stock', 'sma_gap',
    'momentum_3', 'momentum_5', 'macd', 'macd_signal']
X = df[features]
y = df['target']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)
model = XGBRegressor(n_estimators=200, learning_rate=0.05, max_depth=4, random_state=42)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

# --- Step 7: Evaluation ---
spearman_corr, _ = spearmanr(y_test, y_pred)
sharpe_variant = spearman_corr / np.std(y_pred)

# --- Step 8: Streamlit Output ---
st.subheader("📊 Model Performance")
st.write(f"**Spearman Correlation:** {spearman_corr:.4f}")
st.write(f"**Sharpe Ratio Variant:** {sharpe_variant:.4f}")

st.subheader("📉 Prediction vs Actual")
import matplotlib.pyplot as plt
import io
fig, ax = plt.subplots(figsize=(10, 4))
ax.plot(y_test.values, label='Actual')
ax.plot(y_pred, label='Predicted')
ax.set_title('Return Prediction')
ax.legend()
st.pyplot(fig)
st.markdown("-----")
st.markdown(" 📉Devolped by Haider Ali")


2025-08-01 06:02:59.505 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-01 06:02:59.507 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-01 06:02:59.508 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-01 06:02:59.509 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-01 06:02:59.510 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-01 06:02:59.511 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-01 06:02:59.512 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-01 06:02:59.514 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

DeltaGenerator()

In [9]:
import pickle
import os

# Create folder to save model
os.makedirs("saved_model", exist_ok=True)

# Save the trained model to a file
with open("saved_model/xgboost_model.pkl", "wb") as f:
    pickle.dump(model, f)

In [10]:
with open("saved_model/xgboost_model.pkl", "rb") as f:
    loaded_model = pickle.load(f)